In [1]:
# data.sink_parquet(train_fp_out, compression='uncompressed')

In [2]:
import os

os.chdir("..")

In [ ]:
import polars as pl

from src.datatypes import BaseSchema, Filepaths, TrainSchema, TestSchema

In [4]:
filepaths = Filepaths()

# Alias
TS = TrainSchema

train_schema = BaseSchema.__annotations__ | TrainSchema.__annotations__
test_schema = BaseSchema.__annotations__ | TrainSchema.__annotations__ | TestSchema.__annotations__

In [5]:
data = TrainSchema.validate(pl.scan_parquet(filepaths.train, schema=train_schema))
test = TestSchema.validate(pl.scan_csv(filepaths.test, schema=test_schema))

n_rows: int = data.select(pl.len()).collect()["len"][0]
print(f"{n_rows = }")

n_rows_test: int = test.select(pl.len()).collect()["len"][0]
print(f"{n_rows_test = }")
print()

n_null_values = data.null_count().collect()
print(f"{n_null_values = }")
print()

n_unique_rows: int = data.unique().select(pl.len()).collect(engine="streaming")["len"][0]
n_duplicates: int = n_rows - n_unique_rows

print(f"{n_duplicates = }")

n_rows = 184903890
n_rows_test = 18790469

n_null_values = shape: (1, 8)
┌─────┬─────┬────────┬─────┬─────────┬────────────┬─────────────────┬───────────────┐
│ ip  ┆ app ┆ device ┆ os  ┆ channel ┆ click_time ┆ attributed_time ┆ is_attributed │
│ --- ┆ --- ┆ ---    ┆ --- ┆ ---     ┆ ---        ┆ ---             ┆ ---           │
│ u32 ┆ u32 ┆ u32    ┆ u32 ┆ u32     ┆ u32        ┆ u32             ┆ u32           │
╞═════╪═════╪════════╪═════╪═════════╪════════════╪═════════════════╪═══════════════╡
│ 0   ┆ 0   ┆ 0      ┆ 0   ┆ 0       ┆ 0          ┆ 184447044       ┆ 0             │
└─────┴─────┴────────┴─────┴─────────┴────────────┴─────────────────┴───────────────┘

n_duplicates = 3619254


Drop duplicate rows

In [ ]:
data = data.unique()
data.sink_parquet(filepaths.train_unique, compression='uncompressed')